In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.style.use('dark_background')
%matplotlib inline

In [2]:
class RoverMDP():
    def __init__(self):
        self.states = ['low', 'medium', 'high', 'top']
        self.energy_cost = {'slow': 4, 'rapid': 12}

    def startState(self):
        return self.states[0]

    def isEnd(self, state):
        return (state == self.states[3])

    def actions(self, state):
        if state == self.states[3]:
            return []
        return ['slow', 'rapid']

    def NewStateProbReward(self, state, action):
        newStateProbReward = []
        if action in self.actions(state):
            if state == self.states[0]:
                next_state = 1
            elif state == self.states[1]:
                next_state = 2
            elif state == self.states[2]:
                next_state = 3
            else:
                return [(state, 1, 0)]  # terminal state

            if action == 'slow':
                newStateProbReward.append((self.states[next_state], 0.3, -self.energy_cost[action]))
                newStateProbReward.append((self.states[0], 0.7, -self.energy_cost[action]))
            elif action == 'rapid':
                newStateProbReward.append((self.states[next_state], 0.5, -self.energy_cost[action]))
                newStateProbReward.append((self.states[0], 0.5, -self.energy_cost[action]))
        return newStateProbReward

    def discount(self):
        return 0.95

In [3]:
# Create the die game MDP object
roverMdp = RoverMDP()

In [4]:
# What are the possible actions from state 0 (in the game)?
roverMdp.actions(0)

['slow', 'rapid']

---
 Consider the following policies:


* Policy-1: π (slowly | low) = 0.9, π (slowly | medium) = 0.1, π (rapidly | high) = 0.95


* Policy-2: π (rapidly | low) = 0.9, π (slowly | medium) = 0.9, π (slowly | high) = 0.95.


Briefly describe the policies using plain English.

---

In [16]:
def policy(roverMdp, state, policyType='Policy1'):
    if policyType == 'Policy1':
        # Policy-1 specifications:
        # π(slowly | low) = 0.9, π(slowly | medium) = 0.1, π(rapidly | high) = 0.95
        if state == roverMdp.states[0]:  # low state
            action_probs = [0.9, 0.1]  # [slowly, rapidly]
            action = np.random.choice(['slowly', 'rapidly'], p=action_probs)
        elif state == roverMdp.states[1]:  # medium state
            action_probs = [0.1, 0.9]  # [rapidly, slowly]
            action = np.random.choice(['rapidly', 'slowly'], p=action_probs)
        elif state == roverMdp.states[2]:  # high state
            action_probs = [0.05, 0.95]  # [rapidly, slowly]
            action = np.random.choice(['rapidly', 'slowly'], p=action_probs)
        else:
            action = roverMdp.actions(state)[0]

    elif policyType == 'Policy2':
        if state == roverMdp.states[0]:  # low state
            action_probs = [0.9, 0.1]  # ['rapid' , 'slow']
            action = np.random.choice(['rapid' , 'slow'], p=action_probs)
        elif state == roverMdp.states[1]:  # medium state
            action_probs = [0.1, 0.9]  # ['rapid' , 'slow']
            action = np.random.choice(['rapid' , 'slow'], p=action_probs)
        elif state == roverMdp.states[2]:  # high state
            action_probs = [0.05, 0.95]  # ['rapid' , 'slow']
            action = np.random.choice(['rapid' , 'slow'], p=action_probs)
        else:
            action = roverMdp.actions(state)[0]

    return action

'slow'

---
 Calculate the values of the states under both policies above using simulation. Use 0.95
as the discount factor. Which policy is better?

---

In [8]:
import random
import numpy as np

# Create the rover MDP object
roverMDP = RoverMDP()

# Define policies
def policy1(state):
    if state == 'low':
        return 'slow' if random.random() < 0.9 else 'rapid'
    elif state == 'medium':
        return 'slow' if random.random() < 0.1 else 'rapid'
    elif state == 'high':
        return 'rapid' if random.random() < 0.95 else 'slow'
    return None

def policy2(state):
    if state == 'low':
        return 'rapid' if random.random() < 0.9 else 'slow'
    elif state == 'medium':
        return 'slow' if random.random() < 0.9 else 'rapid'
    elif state == 'high':
        return 'slow' if random.random() < 0.95 else 'rapid'
    return None

# Simulation function
def simulate(policy, roverMDP, num_episodes=1000):
    state_values = {state: 0 for state in roverMDP.states}
    for _ in range(num_episodes):
        G = 0  # cumulative reward
        k = 0  # time step
        state = roverMDP.startState()  # start state
        while not roverMDP.isEnd(state):
            action = policy(state)  # action to be taken at the current state
            if action is None:
                break
            SPR_list = roverMDP.NewStateProbReward(state, action)
            prob = [tup[1] for tup in SPR_list]
            index = np.random.choice(range(len(SPR_list)), size=1, p=prob)[0]
            newState = SPR_list[index][0]
            reward = SPR_list[index][2]  # reward
            G = G + (roverMDP.discount())**k * reward
            state = newState
            k += 1
        state_values[state] += G
    for state in state_values:
        state_values[state] /= num_episodes
    return state_values

# Run simulations
valuesPolicy1 = simulate(policy1, roverMDP)
valuesPolicy2 = simulate(policy2, roverMDP)

print("Values under Policy 1:", valuesPolicy1)
print("Values under Policy 1:", valuesPolicy2)

#Determine the better policy

betterPolocy = "Policy 1" if sum(valuesPolicy1.values()) > sum(valuesPolicy2.values()) else "Policy 2"
print(f"Better policy is {betterPolocy}")

Values under Policy 1: {'low': 0.0, 'medium': 0.0, 'high': 0.0, 'top': -70.63696009268645}
Values under Policy 1: {'low': 0.0, 'medium': 0.0, 'high': 0.0, 'top': -116.44532025498033}
Better policy is Policy 1


---
Solve the system of equations using the rref() function of the sympy library. Do
your answers for the values of the states under Policy-1 agree with what you had from
simulation in part (c)?

---

In [19]:
import sympy
from sympy.matrices import Matrix
from sympy import symbols

# Define the matrix and vector
A = Matrix([
    [0.335, -0.0285, 0, 0],
    [-0.665, 0.715, -0.285, 0],
    [0.065, 0, 0.715, 0],
    [0, 0, 0, 1]
])

b = Matrix([-4.6, -11.6, -12, 0])

# Solve the system using rref() method
Ab = A.row_join(b)
rref_Ab, pivots = Ab.rref(iszerofunc=lambda x: abs(x) < 1e-8, simplify=lambda x: x)

x_solution = rref_Ab[:, -1]

print("Solution using rref():", x_solution)


Solution using rref(): Matrix([[-16.9712450857036], [-38.0830562705506], [-15.2403763208801], [0]])
